In [161]:
import os
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from math import floor
from scripts.constants import ATTENTION_LAYER_NAMES, WIN_SHIFT_ATTENTION_LAYER_INDEXES, DOWN_SAMPLE_ATTENTION_LAYER_INDEXES

In [162]:
# Data directories and paths
input_data_dir = 'input_data'
output_data_dir = 'output_data'
data_date = "2018-01-01"
data_time = "12:00"
input_name = "input_surface"

In [163]:
layer_index = 2
head = 5

layer_name = ATTENTION_LAYER_NAMES[layer_index]
if layer_index in DOWN_SAMPLE_ATTENTION_LAYER_INDEXES:
    patchSize = 8
    chunk_size_lat = 48
    chunk_size_lon = 96
else:
    patchSize = 4
    chunk_size_lat = 24
    chunk_size_lon = 48

In [ ]:
# Load input data
input_surface_path = os.path.join(input_data_dir, data_date, data_time, f"{input_name.replace('/', '_')}.npy")
input_surface = np.load(input_surface_path)  

input_upper_path = os.path.join(input_data_dir, data_date, data_time, f"{input_name.replace('/', '_')}.npy")
input_upper = np.load(input_upper_path)  

if layer_index in WIN_SHIFT_ATTENTION_LAYER_INDEXES:
    # roll the data
    pass

In [165]:
path = os.path.join(output_data_dir, data_date, data_time, f"{layer_name.replace('/', '_')}.npy")
attention_output = np.load(path)
attention_output.shape

(15, 64, 12, 144, 144)

In [ ]:
lat_index = 6
lon_index = 1
pl_index = 0

surface_var_index = {
    "MSLP": 0,
    "U10": 1,
    "V10": 2,
    "T2M": 3,
}

weather_var = "T2M"

lat_coord_start = lat_index * chunk_size_lat
lat_coord_end = lat_coord_start + chunk_size_lat
lon_coord_start = lon_index * chunk_size_lon
lon_coord_end = lon_coord_start + chunk_size_lon

lat_coord_start, lat_coord_end, lon_coord_start, lon_coord_end

input_1 = input_data[surface_var_index[weather_var], lat_coord_start:lat_coord_end, lon_coord_start:lon_coord_end]

att_pat = attention_output[lon_index, lat_index + lat_index * pl_index, head, :, :]

In [ ]:
# Create a subplot with two plots
fig = make_subplots(rows=1, cols=2)

# Add the two heatmaps to the figure
trace1 = go.Heatmap(
    z=input_1,
    x0=0, dx=1,
    y0=0, dy=1,
    showscale=False,
    hoverinfo='skip',
    colorscale='Viridis',
    zmin=input_1.min(), zmax=input_1.max(),
)

trace2 = go.Heatmap(
    z=att_pat,
    x0=0, dx=1,
    y0=0, dy=1,
    showscale=False,
    colorscale='Viridis',
    zmin=att_pat.min(), zmax=att_pat.max(),
)

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=2)

# Set up the axes for both subplots
num_rows_input1, num_cols_input1 = input_1.shape
num_rows_att_pat, num_cols_att_pat = att_pat.shape

# Configure axes ranges and reverse y-axis to match array indexing
fig.update_xaxes(range=[0, num_cols_input1], row=1, col=1)
fig.update_yaxes(range=[num_rows_input1, 0], row=1, col=1, scaleratio=1)

fig.update_xaxes(range=[0, num_cols_att_pat], row=1, col=2)
fig.update_yaxes(range=[num_rows_att_pat, 0], row=1, col=2, scaleratio=1)

# Convert the figure to a FigureWidget for interactivity
fig_widget = go.FigureWidget(fig)

# Initialize a rectangle shape for highlighting (hidden initially)
fig_widget.layout.shapes = [
    dict(
        type="rect",
        xref='x1',
        yref='y1',
        x0=0,
        x1=1,
        y0=0,
        y1=1,
        line=dict(color="red", width=2),
        fillcolor="rgba(255, 0, 0, 0.3)",
        layer="above",
        visible=False,
    ),
    dict(
        type="rect",
        xref='x1',
        yref='y1',
        x0=0,
        x1=1,
        y0=0,
        y1=1,
        line=dict(color="green", width=2),
        fillcolor="rgba(0, 255, 0, 0.3)",
        layer="above",
        visible=False,
    )
]

def coords_2_highlight(fig_widget, lat, lon, shape_index):
    # Calculate rectangle coordinates in input_1
    x0 = lon
    x1 = lon + patchSize

    y0 = lat
    y1 = lat + patchSize

    with fig_widget.batch_update():
        # Update rectangle's position and make it visible
        fig_widget.layout.shapes[shape_index].x0 = x0
        fig_widget.layout.shapes[shape_index].x1 = x1
        fig_widget.layout.shapes[shape_index].y0 = y1  # Note inversion due to y-axis reversal
        fig_widget.layout.shapes[shape_index].y1 = y0
        fig_widget.layout.shapes[shape_index].visible = True

# Define the hover callback function
def hover_fn(trace, points, state):
    if points.point_inds:
        # Get the index of the hovered point in the flattened array
        y, x = points.point_inds[0]

        # Compute variables as in your code
        k_pl = math.floor(x / 72)
        q_pl = math.floor(y / 72)

        k_lat = (math.floor(x / 12) % 6) * patchSize
        q_lat = (math.floor(y / 12) % 6) * patchSize

        k_lon = (x % 12) * patchSize
        q_lon = (y % 12) * patchSize

        coords_2_highlight(fig_widget, k_lat, k_lon, 0)
        coords_2_highlight(fig_widget, q_lat, q_lon, 1)


# Define the unhover callback function to hide the rectangle
def unhover_fn(trace, points, state):
    with fig_widget.batch_update():
        fig_widget.layout.shapes[0].visible = False

# Attach the hover and unhover events to the second heatmap
fig_widget.data[1].on_hover(hover_fn)
fig_widget.data[1].on_unhover(unhover_fn)

# Display the interactive figure
fig_widget

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58'],
                             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
              'dx': 1,
              'dy': 1,
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '6ff72c09-d3e2-4e6b-90d3-a358ec68b5e5',
              'x0': 0,
              'xaxis': 'x',
              'y0': 0,
              'yaxis': 'y',
              'z': array([[285.33237, 285.53687, 285.81152, ..., 289.0571 , 289.2569 , 289.565  ],
                          [285.13968, 285.47363, 285.73175, ..., 289.27054, 289.47345, 289.77313],
 